**How to Query the World Bank: International Debt Data (BigQuery Dataset)**

In [1]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
wbid = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="world_bank_intl_debt")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "world_bank_intl_debt")
bq_assistant.list_tables()

['country_series_definitions',
 'country_summary',
 'international_debt',
 'series_summary',
 'series_times']

In [3]:
bq_assistant.head("international_debt", num_rows=15)

,country_name,country_code,indicator_name,indicator_code,year,value
0,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,2021,1553000.0
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,2020,45458000.0
2,Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,2017,8276000.0
3,Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,2018,15646000.0
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,2022,27428000.0
5,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,2022,56988000.0
6,Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,2022,10684000.0
7,Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,2022,94128000.0
8,Afghanistan,AFG,"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD,2020,45458000.0
9,Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,2018,93628000.0


In [4]:
bq_assistant.table_schema("international_debt")

[SchemaField('country_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('country_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('year', 'INTEGER', 'NULLABLE', '', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', '', ())]

What countries had the largest outstanding debt in 2016?


In [5]:
query1 = """
SELECT
  id.country_name,
  id.value AS debt --format in DataStudio
FROM (
  SELECT
    country_code,
    region
  FROM
    `bigquery-public-data.world_bank_intl_debt.country_summary`
  WHERE
    region != "" ) cs --aggregated countries do not have a region
INNER JOIN (
  SELECT
    country_code,
    country_name,
    value,
    year
  FROM
    `bigquery-public-data.world_bank_intl_debt.international_debt`
  WHERE
    indicator_code = "DT.DOD.PVLX.CD"
    AND year = 2016 ) id
ON
  cs.country_code = id.country_code
ORDER BY
  id.value DESC
;
        """
response1 = wbid.query_to_pandas_safe(query1)
response1.head(50)

,country_name,debt
0,Mexico,2.990087e+11
1,Russian Federation,2.004489e+11
2,Indonesia,1.828286e+11
3,Brazil,1.825803e+11
4,India,1.657378e+11
5,China,1.274301e+11
6,Argentina,1.232277e+11
7,Turkey,1.231449e+11
8,Colombia,7.800229e+10
9,"Venezuela, RB",6.270904e+10


![](https://cloud.google.com/bigquery/images/outstanding-debt.png)
https://cloud.google.com/bigquery/images/outstanding-debt.png
